In [1]:
import numpy as np
import os
import seaborn as sns
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from tqdm import tqdm


Bad key "text.kerning_factor" on line 4 in
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ohikendoit/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ohikendoit/opt/anaconda3

In [2]:
class_names = ['dog', 'elephant', 'giraffe', 'guitar', 'horse', 'house', 'person']
class_names_label = {class_name: i for i, class_name in enumerate(class_names)}

nb_classes = len(class_names)
IMAGE_SIZE = (150, 150)

In [7]:
def load_data():
    datasets = ['/Users/ohikendoit/Desktop/2021 Dev-Matching ML Task/train', '/Users/ohikendoit/Desktop/2021 Dev-Matching ML Task/test']
    output = []
    
    for dataset in datasets:
        images = []
        labels = []
        
        print("Loading {}".format(dataset))
        
        for folder in os.listdir(dataset):
            if not folder.startswith('.'):
                label = class_names_label[folder]

                for file in tqdm(os.listdir(os.path.join(dataset, folder))):
                    img_path = os.path.join(os.path.join(dataset,folder), file)

                    image = cv2.imread(img_path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    image = cv2.resize(image, IMAGE_SIZE)

                    images.append(image)
                    labels.append(label)

            images = np.array(images, dtype = 'float32')
            labels = np.array(labels, dtype = 'int32')

            output.append((images, labels))
    
    return output

In [8]:
(train_images, train_labels), (test_images, test_labels) = load_data()

  0%|          | 0/329 [00:00<?, ?it/s]

Loading /Users/ohikendoit/Desktop/2021 Dev-Matching ML Task/train


AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
train_images, train_labels = shuffle(train_images, train_labels, random_state=20)

In [ ]:
#Basic Model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu', input_shape = (150, 150, 3)), 
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3, 3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(6, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer = 'adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(train_images, train_labels, batch_size=128, epochs=20, validation_split = 0.2)

In [ ]:

def plot_accuracy_loss(history):
    """
        Plot the accuracy and the loss during the training of the nn.
    """
    fig = plt.figure(figsize=(10,5))

    # Plot accuracy
    plt.subplot(221)
    plt.plot(history.history['acc'],'bo--', label = "acc")
    plt.plot(history.history['val_acc'], 'ro--', label = "val_acc")
    plt.title("train_acc vs val_acc")
    plt.ylabel("accuracy")
    plt.xlabel("epochs")
    plt.legend()

    # Plot loss function
    plt.subplot(222)
    plt.plot(history.history['loss'],'bo--', label = "loss")
    plt.plot(history.history['val_loss'], 'ro--', label = "val_loss")
    plt.title("train_loss vs val_loss")
    plt.ylabel("loss")
    plt.xlabel("epochs")

    plt.legend()
    plt.show()

In [ ]:
plot_accuracy_loss(history)

In [ ]:
test_loss = model.evaluate(test_images, test_labels)


In [ ]:
import pandas as pd
submission = pd.read_csv('test_answer_sample_.csv')
submission['answer value'] = pred_idx
del submission['Unnamed: 0']
submission.to_csv('result.csv')